In [ ]:
!pip install dremio-simple-query

## Get Dremio Access Token

In [17]:
from dremio_simple_query.connect import get_token

## URL to Login Endpoint
login_endpoint = "http://localhost:9047/apiv2/login"

## Payload for Login
## Update with your Dremio credentials
payload = {
    "userName": "username",
    "password": "password"
}

## Get token from API
token = get_token(uri = login_endpoint, payload=payload)

## Load product_reviews as a pandas DataFrame
- Query Dremio from Python using PyArrow flight
- Load the query results to a Pandas DataFrame

In [ ]:
from pyarrow import flight

# Connect to Dremio using Flight
client = flight.FlightClient(f"grpc+tcp://localhost:32010")

# Authenticate
headers = [
    (b'authorization', f"bearer {token}".encode('utf-8'))
]

# Query to get all product reviews
sql = "SELECT * FROM catalog.bronze.ecoride.product_reviews"

# Create a Flight descriptor
descriptor = flight.FlightDescriptor.for_command(sql)

# Perform the query using call_options to include headers
options = flight.FlightCallOptions(headers=headers)
flight_info = client.get_flight_info(descriptor, options=options)
reader = client.do_get(flight_info.endpoints[0].ticket, options=options)

# Convert to Pandas DataFrame
product_reviews = reader.read_pandas()

# Display the DataFrame
print(product_reviews.head())

## Basic Statistics
- Run basic statistics on the product_reviews DataFrame

In [ ]:
print(product_reviews.describe())
print(product_reviews['VehicleModel'].value_counts())

## Advanced Analytics
Run advanced analytics and machine learning on the product_reviews Dataset:

1. Wordclouds per vehicle model
2. Reviews sentiment analysis
3. Sentiment over time and sentiment distribution accros several categories

### Wordclouds per vehicle model

In [ ]:
!pip install wordcloud

In [ ]:
from wordcloud import WordCloud
import matplotlib.pyplot as plt
import re

# List of vehicle models to exclude
vehicle_models = product_reviews['VehicleModel'].unique().tolist()

# Function to preprocess and clean text
def preprocess_text(text, vehicle_models):
    for model in vehicle_models:
        text = re.sub(model, '', text, flags=re.IGNORECASE)
    return text.lower()

# Apply the preprocessing function
product_reviews['CleanReviewText'] = product_reviews['ReviewText'].apply(lambda x: preprocess_text(x, vehicle_models))

# Function to create a word cloud
def create_word_cloud(text, title):
    wordcloud = WordCloud(width=800, height=400, background_color='white').generate(text)
    plt.figure(figsize=(10, 5))
    plt.imshow(wordcloud, interpolation='bilinear')
    plt.title(title)
    plt.axis('off')
    plt.show()

# Aggregate review text by vehicle model
grouped_reviews = product_reviews.groupby('VehicleModel')['CleanReviewText'].apply(' '.join)

# Create word clouds for each vehicle model, excluding the vehicle name
for model, text in grouped_reviews.items():
    create_word_cloud(text, f'Word Cloud for {model}')

### Sentiment analysis per vehicle model

In [ ]:
!pip install textblob

In [ ]:
from textblob import TextBlob
import pandas as pd
import matplotlib.pyplot as plt

# Sentiment analysis using TextBlob
def get_sentiment(text):
    blob = TextBlob(text)
    sentiment = blob.sentiment.polarity
    if sentiment > 0:
        return 'positive'
    elif sentiment < 0:
        return 'negative'
    else:
        return 'neutral'

product_reviews['Sentiment'] = product_reviews['ReviewText'].apply(get_sentiment)

# Distribution of sentiments
sentiment_counts = product_reviews['Sentiment'].value_counts()
sentiment_counts.plot(kind='bar', figsize=(10, 6), colormap='viridis')
plt.title('Sentiment Distribution')
plt.xlabel('Sentiment')
plt.ylabel('Count')
plt.show()

# Distribution of sentiments per vehicle model
sentiment_per_model = product_reviews.groupby('VehicleModel')['Sentiment'].value_counts().unstack().fillna(0)
sentiment_per_model.plot(kind='bar', stacked=True, figsize=(10, 6), colormap='viridis')
plt.title('Sentiment Distribution per Vehicle Model')
plt.xlabel('Vehicle Model')
plt.ylabel('Count')
plt.show()

### Setiment over time

In [ ]:
product_reviews['Date'] = pd.to_datetime(product_reviews['Date'])
sentiment_over_time = product_reviews.groupby([product_reviews['Date'].dt.to_period('M')])['Sentiment'].value_counts().unstack().fillna(0)
sentiment_over_time.plot(kind='line', subplots=True, layout=(2, 2), figsize=(15, 10), title="Sentiment Over Time")
plt.show()

### Get customer_segmentation dataset from the Lakehouse
- Load `customer_segmentation` as a Pandas DataFrame
- Join `customer_segmentation` with `product_reviews` on `customer_id`

In [ ]:
# Query to get all product reviews
sql = "SELECT * FROM lakehouse.gold.customer_segmentation"

# Create a Flight descriptor
descriptor = flight.FlightDescriptor.for_command(sql)

# Perform the query using call_options to include headers
options = flight.FlightCallOptions(headers=headers)
flight_info = client.get_flight_info(descriptor, options=options)
reader = client.do_get(flight_info.endpoints[0].ticket, options=options)

# Convert to Pandas DataFrame
customer_segmentation = reader.read_pandas()

# Display the DataFrame
print(customer_segmentation.head())

In [ ]:
# Ensure customer_id is of type integer
customer_segmentation['customer_id'] = customer_segmentation['customer_id'].astype(int)
product_reviews['CustomerID'] = product_reviews['CustomerID'].astype(int)

# Convert CustomerID to customer_id for merging
product_reviews.rename(columns={'CustomerID': 'customer_id'}, inplace=True)

# Merge the datasets on customer_id
merged_df = pd.merge(product_reviews, customer_segmentation, on='customer_id')

# Display the merged DataFrame
print(merged_df.head())

### Sentiment by State

In [ ]:
sentiment_by_state = merged_df.groupby('state')['Sentiment'].value_counts().unstack().fillna(0)
sentiment_by_state.plot(kind='bar', stacked=True, figsize=(10, 6), colormap='viridis')
plt.title('Sentiment Distribution by State')
plt.xlabel('State')
plt.ylabel('Count')
plt.show()

### Sentiment by VIP status

In [ ]:
# Add is_vip column based on total_purchases
merged_df['is_vip'] = merged_df['total_purchases'] > 3
sentiment_by_vip = merged_df.groupby('is_vip')['Sentiment'].value_counts().unstack().fillna(0)
sentiment_by_vip.plot(kind='bar', stacked=True, figsize=(10, 6), colormap='viridis')
plt.title('Sentiment Distribution by VIP Status')
plt.xlabel('VIP Status')
plt.ylabel('Count')
plt.show()